# Video emotions Classification

In [1]:
!dir

dataset  FootballVideoClassification.ipynb  test.csv  tmp  train.csv


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('dataset/train')

label_types = os.listdir('dataset/train')
print (label_types) 

['Goal', 'Happy', 'Loss']


# Training Data Preparation

In [3]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('dataset/train' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('dataset/train' + '/' +item) + '/' + room))
    
# Build a dataframe        
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

    tag                     video_name
0  Goal  dataset/train/Goal/Goal10.MP4
1  Goal   dataset/train/Goal/Goal1.mp4
2  Goal   dataset/train/Goal/Goal2.MP4
3  Goal   dataset/train/Goal/Goal4.mp4
4  Goal   dataset/train/Goal/Goal3.mp4
     tag                    video_name
19  Loss  dataset/train/Loss/Loss4.mp4
20  Loss  dataset/train/Loss/Loss6.mp4
21  Loss  dataset/train/Loss/Loss1.mp4
22  Loss  dataset/train/Loss/Loss5.mp4
23  Loss  dataset/train/Loss/Loss3.mp4


In [4]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

# Test Data Preparation

In [5]:
dataset_path = os.listdir('dataset/test')
print(dataset_path)

room_types = os.listdir('dataset/test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('dataset/test' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('dataset/test' + '/' +item) + '/' + room))
    
# Build a dataframe        
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')

['Goal', 'Happy', 'Loss']
Types of activities found:  3
    tag                   video_name
0  Goal  dataset/test/Goal/Goal2.mp4
1  Goal  dataset/test/Goal/Goal5.mp4
2  Goal  dataset/test/Goal/Goal1.mp4
3  Goal  dataset/test/Goal/Goal4.mp4
4  Goal  dataset/test/Goal/Goal3.mp4
     tag                   video_name
10  Loss  dataset/test/Loss/Loss2.mp4
11  Loss  dataset/test/Loss/Loss4.mp4
12  Loss  dataset/test/Loss/Loss1.mp4
13  Loss  dataset/test/Loss/Loss5.mp4
14  Loss  dataset/test/Loss/Loss3.mp4


# Installing tesorflow

In [6]:
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-qedm6ckw
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-qedm6ckw
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0.dev0-py3-none-any.whl size=180332 sha256=251bd9b59569b625636df45ce23718780b5d282ea85b8fe5684b06b93d191cfc
  Stored in directory: /tmp/pip-ephem-wheel-cache-0vzm2n6o/wheels/3b/ee/a2/ab4d36a9a4af495bcb936f3e849d4b497b65fa40548a68d6c3
Successfully built tensorflow-docs


In [7]:
!pip install tensorflow

In [8]:
!pip install imutils
!pip install opencv-python
!pip install imageio


In [9]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

2023-02-06 23:35:47.970046: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 23:35:48.125455: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-06 23:35:48.125483: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-06 23:35:48.773561: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# Data preparation

In [10]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 24
Total videos for testing: 15


,Unnamed: 0,video_name,tag
12,12,dataset/train/Happy/Happy1.mp4,Happy
17,17,dataset/train/Happy/Happy7.mp4,Happy
23,23,dataset/train/Loss/Loss3.mp4,Loss
16,16,dataset/train/Happy/Happy2.mp4,Happy
2,2,dataset/train/Goal/Goal2.MP4,Goal
11,11,dataset/train/Happy/Happy4.mp4,Happy
9,9,dataset/train/Happy/Happy5.mp4,Happy
20,20,dataset/train/Loss/Loss6.mp4,Loss
22,22,dataset/train/Loss/Loss5.mp4,Loss
5,5,dataset/train/Goal/Goal6.mp4,Goal


# Adding video to the network

In [11]:
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

# Feature Extraction

In [12]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

2023-02-06 23:36:11.039332: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/user/.local/lib/python3.8/site-packages/cv2/../../lib64:
2023-02-06 23:36:11.039365: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-06 23:36:11.039392: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (user-Vostro-3578): /proc/driver/nvidia/version does not exist
2023-02-06 23:36:11.039650: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate co

In [13]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['Goal', 'Happy', 'Loss']


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2]])

In [14]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [15]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values
    
    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")


Frame features in train set: (24, 20, 2048)
Frame masks in train set: (24, 20)
train_labels in train set: (24, 1)
test_labels in train set: (15, 1)


In [16]:
 #Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
1/1 [==============================] - ETA: 0s - loss: 1.0986 - accuracy: 0.5625
Epoch 1: val_loss improved from inf to 1.09945, saving model to ./tmp/video_classifier
1/1 [==============================] - 10s 10s/step - loss: 1.0986 - accuracy: 0.5625 - val_loss: 1.0994 - val_accuracy: 0.0000e+00
Epoch 2/30
1/1 [==============================] - ETA: 0s - loss: 1.0979 - accuracy: 0.5625
Epoch 2: val_loss did not improve from 1.09945
1/1 [==============================] - 0s 64ms/step - loss: 1.0979 - accuracy: 0.5625 - val_loss: 1.1003 - val_accuracy: 0.0000e+00
Epoch 3/30
1/1 [==============================] - ETA: 0s - loss: 1.0973 - accuracy: 0.5625
Epoch 3: val_loss did not improve from 1.09945
1/1 [==============================] - 0s 68ms/step - loss: 1.0973 - accuracy: 0.5625 - val_loss: 1.1011 - val_accuracy: 0.0000e+00
Epoch 4/30
1/1 [==============================] - ETA: 0s - loss: 1.0966 - accuracy: 0.5625
Epoch 4: val_loss did not improve from 1.09945
1/1 [===

1/1 [==============================] - 0s 26ms/step - loss: 1.0986 - accuracy: 0.3333
Test accuracy: 33.33%


In [18]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: dataset/test/Goal/Goal5.mp4
1/1 [==============================] - 0s 24ms/step
  Happy: 33.36%
  Goal: 33.36%
  Loss: 33.29%


In [19]:
from IPython.display import HTML
HTML("""
<video width="520" height="440" controls>
  <source src="dataset/test/Goal/Goal5.mp4" type="video/mp4">
</video>
""")